In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import tensorflow as tf

2024-08-28 12:56:13.836937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 12:56:13.853231: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 12:56:13.858143: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 12:56:13.871113: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-28 12:56:14.751290: W tensorflow/comp

In [3]:
batch_size = 64
epochs = 200
latent_dim = 256
num_samples = 10000

In [4]:
data_path = 'dataset/fra.txt'

In [5]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, target_text,_= line.split('\t')
    target_text = '\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
input_texts

['Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I know.',
 'I left.',
 'I left.',
 'I lied.',
 'I lost.',
 'I paid.',
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?',
 'Thanks.',
 'We try.',
 'We won.',
 'We won.',
 'We won.',
 'We won.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be calm.',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fai

In [7]:
with open('output.txt', 'w') as file:
    for item in input_texts:
        file.write(f"{item}\n")

In [8]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_len = max([len(txt) for txt in input_texts])
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [9]:
print('num of samples:', len(input_texts))
print('num of unique input tokens:', num_encoder_tokens)
print('num of unique output tokens:', num_decoder_tokens)
print('max seq len for inputs:', max_encoder_seq_len)
print('max seq len for outputs:', max_decoder_seq_len)

num of samples: 10000
num of unique input tokens: 71
num of unique output tokens: 92
max seq len for inputs: 16
max seq len for outputs: 59


In [10]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)]
)

In [11]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_len, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),    
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),
    dtype = 'float32')

In [18]:
encoder_input_data.shape

(10000, 16, 71)

In [13]:
#One hot encoding
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]]=1
        if t>0:
            decoder_target_data[i, t-1, target_token_index[char]]=1
    decoder_input_data[i,t+1:, target_token_index[' ']] = 1
    decoder_target_data[i, t:, target_token_index[' ']]=1

In [19]:
encoder_input_data[0].shape

(16, 71)

# Model

In [15]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [16]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [17]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
         batch_size = batch_size,
         epochs=epochs,
         validation_split=0.2)

Epoch 1/200


2024-08-17 13:58:44.202117: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.6926 - loss: 1.5994 - val_accuracy: 0.6939 - val_loss: 1.1289
Epoch 2/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7333 - loss: 1.0026 - val_accuracy: 0.7144 - val_loss: 1.0249
Epoch 3/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7534 - loss: 0.8906 - val_accuracy: 0.7156 - val_loss: 0.9957
Epoch 4/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7773 - loss: 0.7907 - val_accuracy: 0.7638 - val_loss: 0.8191
Epoch 5/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7956 - loss: 0.7113 - val_accuracy: 0.7755 - val_loss: 0.7634
Epoch 6/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8079 - loss: 0.6629 - val_accuracy: 0.7908 - val_loss: 0.7199
Epoch 7/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8144 - loss: 0.6326 - val_accuracy: 0.7901 - val_loss: 0.7151
Epoch 8/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8210 - loss: 0.6113 - val_accuracy

In [20]:
encoder_model = Model(encoder_inputs,encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state= decoder_states_inputs)
decoder_states=[state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs]+decoder_states_inputs, [decoder_outputs]+decoder_states)


In [21]:
reverse_input_char_index= dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index= dict((i, char) for char, i in target_token_index.items())

In [22]:
def decode_sequence(input_seq):
    
    states_value = encoder_model.predict(input_seq, verbose=0)
    
    target_seq = np.zeros((1,1, num_decoder_tokens))
    
    target_seq[0, 0, target_token_index['\t']]=1.
    
    stop_condition= False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h,c = decoder_model.predict([target_seq]+states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0,-1,:])
       
        sampled_char = reverse_target_char_index[sampled_token_index]
        
        decoded_sentence += sampled_char
        if (sampled_char=='\n' or len(decoded_sentence)>max_decoder_seq_len):
            stop_condition=True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index]=1.
        
        states_value = [h, c]
    return decoded_sentence

for seq_index in range(100):
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decode_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence  :', input_texts[seq_index])
    print('Decoded Sentence:', decode_sentence)

-
Input sentence  : Go.
Decoded Sentence: Va !

-
Input sentence  : Hi.
Decoded Sentence: Salut.

-
Input sentence  : Hi.
Decoded Sentence: Salut.

-
Input sentence  : Run!
Decoded Sentence: Sarde donc !

-
Input sentence  : Run!
Decoded Sentence: Sarde donc !

-
Input sentence  : Who?
Decoded Sentence: Qui ?

-
Input sentence  : Wow!
Decoded Sentence: Ça alors !

-
Input sentence  : Fire!
Decoded Sentence: Au feu !

-
Input sentence  : Help!
Decoded Sentence: Salue-le cons-lains.

-
Input sentence  : Jump.
Decoded Sentence: Saute.

-
Input sentence  : Stop!
Decoded Sentence: Arrête-toi !

-
Input sentence  : Stop!
Decoded Sentence: Arrête-toi !

-
Input sentence  : Stop!
Decoded Sentence: Arrête-toi !

-
Input sentence  : Wait!
Decoded Sentence: Attends !

-
Input sentence  : Wait!
Decoded Sentence: Attends !

-
Input sentence  : Go on.
Decoded Sentence: Continuez.

-
Input sentence  : Go on.
Decoded Sentence: Continuez.

-
Input sentence  : Go on.
Decoded Sentence: Continuez.

-
Inpu